In [20]:
import cv2
import numpy as np
import random as r

from keras.models import load_model
import keras
from keras.preprocessing.image import load_img, img_to_array

# 좌표용 전역변수 설정
old_x = old_y = -1
# 모델 불러오기
model = load_model("IMS_CNN_test.h5")
# 모델이 예측한 정답
choiced = ""

# 이미지 예측
def predict_image(path):
    global choiced
    img = load_img(path, target_size=(128, 128), color_mode='grayscale')
    x = img_to_array(img) / 255.0
    x = np.reshape(x, (1, 128, 128, 1))
    pred = model.predict(x)
    print("모델 출력값:", pred[0])
    if pred >= 0.5:
        choiced = "X!!"
    else:
        choiced = "O!!"


# 마우스 이벤트(선 그리기)
def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        global old_x, old_y # 전역변수로 받아오지 않으면 사용할 수 없음
        old_x, old_y = x, y # x, y는 마우스에서 받아온 좌표
    elif event == cv2.EVENT_MOUSEMOVE: # 마우스가 움직일 때 발생
        if flags & cv2.EVENT_FLAG_LBUTTON: # ==를 쓰면 다른 키도 입력될수 있음
            cv2.line(img, (old_x, old_y), (x, y), (0, 0, 0), 4, cv2.LINE_AA)
            cv2.imshow(window_name, img)
            old_x, old_y = x, y # 이전 좌표를 업데이트 안하면 중앙에서 마우스 위치로 직선이 그려짐

window_name = "Draw O/X - Save: S, Reset: R, ESC: Exit"
path = 'Userdata/user_data.jpg'

# 데이터셋에 맞게 300*300, 흰색 배경의 캔버스
img = np.ones((300, 300, 3), dtype=np.uint8) * 255
dest_img = img.copy() # 깊은 복사여야만 사용 가능

cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, on_mouse, img)

cv2.imshow(window_name, img)

print("ESC: 종료, S: 저장, R: 리셋")
while(True):   
    # 마우스 이벤트가 발생하면 on_mouse 함수 실행
    # cv2.imshow(window_name, img)
    key = cv2.waitKey()
    if key == ord('s'):
        cv2.imwrite(path, img)
        cv2.destroyAllWindows()
        print("Saved to Userdata/user_data.jpg")
        predict_image(path)
        cv2.imshow(f"Your draw is {choiced}", img)
        key = cv2.waitKey()
        if key == ord('r'):
            break

    elif key == ord('r'):
        img = dest_img.copy()
        cv2.imshow(window_name, img)

    # Esc 키코드: 27
    elif key == 27:
        break
        
cv2.destroyAllWindows()

ESC: 종료, S: 저장, R: 리셋
Saved to Userdata/user_data.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
모델 출력값: [0.02757308]
